In [11]:
import pandas as pd
from pprint import pprint 
import time
import requests as req
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
def init_browser():
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

## Mars Hemispheres scraping

In [13]:
browser = init_browser()
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\rtear\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


In [15]:
hemi_scrape = []
for i in range(4):
    hemi_html = browser.html
    hemi_soup = bs(hemi_html, "html.parser")

    title = hemi_soup.find_all("h3")[i].get_text()
    browser.find_by_tag('h3')[i].click()

    hemi_html = browser.html
    hemi_soup = bs(hemi_html, "html.parser")

    hemi_url = hemi_soup.find("a", text="Sample")["href"]

    # append title and image url to dictionary
    hemi_scrape.append({
        "title": title,
        "image_url": hemi_url
    })
    
    browser.back()

In [17]:
# print the hemispheres dictionary
pprint(hemi_scrape)

[{'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Download'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
